# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2
from PIL import Image
from numpy import expand_dims

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import keras
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 20

In [4]:
def createList(r1, r2): 
    return [item for item in range(r1, r2+1)]

In [5]:
def crop_image(image):     
    # Cropping non symmetric frames
    if image.shape[0] != image.shape[1]:
        image=image[0:120,20:140]
    return image

In [6]:
def resize_image(image,nb_rows,nb_cols):        
    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
    return resized

In [7]:
#we are using batch normalization techniques in the model instead
def normalize_image(image):
    #norm_image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
    norm_image = image - np.min(image)/np.max(image) - np.min(image)
    return image/127.5-1

In [8]:
# Parameters initialization
nb_rows = 120  
nb_cols = 120 
nb_frames = 30  
nb_channel = 3# number of channels in images 3 for color(RGB) and 1 for Gray
batch_size=10
nb_labels = 5

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [9]:

def b2g_image_generator(source_path, folder_list, batch_size):
    batchnum_lst=[]    
    img_idx = createList(0,nb_frames-1) 
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,nb_labels)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    batchnum_lst.append(t[folder + (batch*batch_size)])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #normalize check discussion forum for cropping and resizing 


                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    cropped = crop_image(gray)
                    resized = resize_image(cropped,nb_rows,nb_cols)                
                    batch_img = expand_dims(resized, axis=2)

                    batch_data[folder,idx,:,:,:] = batch_img
    #               batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
    #               batch_data[folder,idx,:,:,2] = norm_image[:,:,2]

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
       
        diff_lst = list(set(t.tolist()).difference(set(batchnum_lst)))
        rem_batch_size = len(diff_lst)
        
        if rem_batch_size > 0:
            
            batch_data = np.zeros((rem_batch_size,nb_frames,nb_rows,nb_cols,nb_channel))
            batch_labels = np.zeros((rem_batch_size,nb_labels))

            for folder_idx,folder in enumerate(diff_lst):

                file_dir = folder.split(';')[0]           
                label = int(folder.split(';')[2])
                imgs = os.listdir(source_path + '/' + file_dir)

                for idx,img in enumerate(imgs):

                    image = imread(source_path + '/' + file_dir +'/'+ img).astype(np.float32)
                    
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    cropped = crop_image(gray)
                    resized = resize_image(cropped,nb_rows,nb_cols)                
                    batch_img = expand_dims(resized, axis=2)
                    
                    batch_data[folder_idx,idx,:,:,:]=batch_img

                batch_labels[folder_idx,label]=1
                
            yield batch_data,batch_labels 

In [10]:

def rgb_image_generator(source_path, folder_list, batch_size):
    batchnum_lst=[]    
    img_idx = createList(0,nb_frames-1)    
    print('folder_lst ',len(folder_list))
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,nb_labels)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    batchnum_lst.append(t[folder + (batch*batch_size)])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #normalize check discussion forum for cropping and resizing 


                    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    cropped = crop_image(image)
                    resized = resize_image(cropped,nb_rows,nb_cols)                
                    #batch_img = expand_dims(resized, axis=2)

                    batch_data[folder,idx,:,:,:] = resized
    #               batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
    #               batch_data[folder,idx,:,:,2] = norm_image[:,:,2]

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
       
        diff_lst = list(set(t.tolist()).difference(set(batchnum_lst)))
        rem_batch_size = len(diff_lst)
        
        if rem_batch_size > 0:
            
            batch_data = np.zeros((rem_batch_size,nb_frames,nb_rows,nb_cols,nb_channel))
            batch_labels = np.zeros((rem_batch_size,nb_labels))

            for folder_idx,folder in enumerate(diff_lst):

                file_dir = folder.split(';')[0]           
                label = int(folder.split(';')[2])
                imgs = os.listdir(source_path + '/' + file_dir)

                for idx,img in enumerate(imgs):

                    image = imread(source_path + '/' + file_dir +'/'+ img).astype(np.float32)
                    cropped = crop_image(image)
                    resized = resize_image(cropped,nb_rows,nb_cols)
                    #batch_img = expand_dims(resized, axis=2)
                    batch_data[folder_idx,idx,:,:,:]=resized

                batch_labels[folder_idx,label]=1
           
            yield batch_data,batch_labels 

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [11]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)


# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [12]:
from keras.models import Sequential,Model
from keras.layers import Dense,GRU,LSTM,Flatten,TimeDistributed,Flatten,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D,MaxPooling3D
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

#write your model here

In [13]:
#conv 3d with batch normalization

def build_conv3d_model():
    
    nb_filters = [8,16,32,64]
    nb_dense = [256, 128, 5]

    # Input
    input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

    # Define model
    model = Sequential()

    model.add(Conv3D(nb_filters[0], 
                     kernel_size=(3,3,3), 
                     strides=(1,1,1),                 
                     input_shape=input_shape,
                     padding='same'))

    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(nb_filters[1], 
                     kernel_size=(3,3,3),
                     strides=(1,1,1),
                     padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(nb_filters[2], 
                     kernel_size=(1,3,3), 
                     strides=(1,1,1),
                     padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(nb_filters[3], 
                     kernel_size=(1,3,3), 
                     strides=(1,1,1),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

    #Flatten Layers
    model.add(Flatten())

    model.add(Dense(nb_dense[0], activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_dense[1], activation='relu'))
    model.add(Dropout(0.5))

    #softmax layer
    model.add(Dense(nb_dense[2], activation='softmax'))
    
    return model
    

In [14]:
def build_conv2d_rnn_model():
    
    nb_filters = [8,16,32,64]
    nb_dense = [256, 128, 5]
    nb_rnn=[16,8]

    # Input
    input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)


    # Define model
    model = Sequential()

    model.add(TimeDistributed(Conv2D(
                     nb_filters[0], 
                     kernel_size=(3,3), 
                     strides=(1,1),   
                     padding='same'),
                     input_shape=input_shape,
                     ))

    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation('relu')))

    model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

    model.add(TimeDistributed(Conv2D(
                     nb_filters[1], 
                     kernel_size=(3,3),
                     strides=(1,1),
                     padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation('relu')))

    model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

    model.add(TimeDistributed(Conv2D(
                     nb_filters[2], 
                     kernel_size=(3,3), 
                     strides=(1,1),
                     padding='same')))

    model.add(TimeDistributed(BatchNormalization()))        
    model.add(TimeDistributed(Activation('relu')))

    model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

    # cnn.add(Conv2D(nb_filters[3], 
    #                  kernel_size=(1,3,3), 
    #                  strides=(1,1,1),
    #                  padding='same'))
    # cnn.add(Activation('relu'))
    # cnn.add(Dropout(0.25))

    # cnn.add(MaxPooling2D(pool_size=(2,2,2)))

    model.add(TimeDistributed(Conv2D(
                     nb_filters[2], 
                     kernel_size=(3,3), 
                     strides=(1,1),
                     padding='same')))        

    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

    # model.add(TimeDistributed(Conv2D(
    #                  nb_filters[2], 
    #                  kernel_size=(3,3), 
    #                  strides=(1,1),
    #                  padding='same')))        

    # model.add(TimeDistributed(Activation('relu')))
    # model.add(TimeDistributed(Dropout(0.25)))

    # model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

    #Flatten Layers
    model.add(TimeDistributed(Flatten()))

    model.add(GRU(nb_rnn[1], return_sequences=False, dropout=0.5))

    model.add(Dense(nb_dense[0], activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_dense[1], activation='relu'))
    model.add(Dropout(0.5))

    #softmax layer
    model.add(Dense(nb_dense[2], activation='softmax'))
    
    return model


In [15]:
def build_vgg16_rnn_model():
    
    def build_vgg_16(in_shape,split_at=8):
        
        vgg = VGG16(weights="imagenet", input_shape = in_shape, include_top = False)    
        for layer in vgg.layers[:split_at]: layer.trainable = False
        for layer in vgg.layers[split_at:]: layer.trainable = True
            
        return vgg
    
    nb_filters = [8,16,32,64]
    nb_dense = [256, 128, 5]
    nb_rnn=[256,8]

    vgg_16_model = build_vgg_16((nb_rows,nb_cols,nb_channel))
    # Input
    input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

    # Define model
    model = Sequential()

    model.add(TimeDistributed(vgg_16_model,
                     input_shape=input_shape,
                     ))
    
    model.add(TimeDistributed(GlobalAveragePooling2D()))

    model.add(TimeDistributed(Flatten()))

    model.add(GRU(nb_rnn[0], return_sequences=False, dropout=0.5))

    model.add(Dense(nb_dense[0], activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_dense[1], activation='relu'))
    model.add(Dropout(0.5))

    #softmax layer
    model.add(Dense(nb_dense[2], activation='softmax'))
    
    return model

In [16]:
def build_vgg19_rnn_model():
    
    def build_vgg_19(in_shape,split_at=11):
        
        vgg = VGG19(weights="imagenet", input_shape = in_shape, include_top = False)    
        for layer in vgg.layers[:split_at]: layer.trainable = False
        for layer in vgg.layers[split_at:]: layer.trainable = True
            
        return vgg
    
    nb_filters = [8,16,32,64]
    nb_dense = [256, 128, 5]
    nb_rnn=[256,8]

    vgg_19_model = build_vgg_19((nb_rows,nb_cols,nb_channel))
    # Input
    input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

    # Define model
    model = Sequential()

    model.add(TimeDistributed(vgg_19_model,
                     input_shape=input_shape,
                     ))
    
    model.add(TimeDistributed(GlobalAveragePooling2D()))

    model.add(TimeDistributed(Flatten()))

    model.add(GRU(nb_rnn[0], return_sequences=False, dropout=0.5))

    model.add(Dense(nb_dense[0], activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_dense[1], activation='relu'))
    model.add(Dropout(0.5))

    #softmax layer
    model.add(Dense(nb_dense[2], activation='softmax'))
    
    return model

In [17]:
def build_resnet50_rnn_model(in_shape,split_at=140):
    
    def build_resnet50_model(in_shape,split_at=140):
    
    # Get base model: ResNet50 
        resnet_model = ResNet50(weights='imagenet', include_top=False,input_shape = in_shape,)    
        # freeze the layers in base model    
        for layer in base_model.layers[:split_at]: layer.trainable = False
        for layer in base_model.layers[split_at:]: layer.trainable = True
        
        return resnet_model
    
    nb_filters = [8,16,32,64]
    nb_dense = [256, 128, 5]
    nb_rnn=[16,8]

    resnet50_model = build_resnet50_model((nb_rows,nb_cols,nb_channel))
    # Input
    input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

    # Define model
    model = Sequential()

    model.add(TimeDistributed(resnet50_model,
                     input_shape=input_shape,
                     ))
    
    model.add(TimeDistributed(GlobalAveragePooling2D()))

    model.add(TimeDistributed(Flatten()))

    model.add(GRU(nb_rnn[1], return_sequences=False, dropout=0.5))

    model.add(Dense(nb_dense[0], activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_dense[1], activation='relu'))
    model.add(Dropout(0.5))

    #softmax layer
    model.add(Dense(nb_dense[2], activation='softmax'))
    
    return model

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [18]:
def get_optimizer(optimizer='adam'):
    if optimizer=='adam':
        return optimizers.Adam()
    elif optimizer =='sgd':
        return optimizers.SGD()      

In [19]:
model = build_conv3d_model()
optimiser = get_optimizer(optimizer='adam')

In [20]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [21]:
if nb_channel==1:
    train_generator = b2g_image_generator(train_path, train_doc, batch_size)
    val_generator = b2g_image_generator(val_path, val_doc, batch_size)
elif nb_channel==3:
    train_generator = rgb_image_generator(train_path, train_doc, batch_size)
    val_generator = rgb_image_generator(val_path, val_doc, batch_size)
    

In [22]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [23]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [24]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


65/67 [============================>.] - ETA: 5s - loss: 1.9284 - categorical_accuracy: 0.2400

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


67/67 [==============================] - 170s 3s/step - loss: 1.9177 - categorical_accuracy: 0.2388 - val_loss: 1.5265 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2019-10-2804_33_03.045348/model-00001-1.92126-0.24133-1.52653-0.31000.h5
Epoch 2/20
67/67 [==============================] - 48s 715ms/step - loss: 1.4652 - categorical_accuracy: 0.3448 - val_loss: 1.3728 - val_categorical_accuracy: 0.4600

Epoch 00002: saving model to model_init_2019-10-2804_33_03.045348/model-00002-1.46524-0.34478-1.37282-0.46000.h5
Epoch 3/20
67/67 [==============================] - 49s 732ms/step - loss: 1.3703 - categorical_accuracy: 0.4030 - val_loss: 1.2794 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2019-10-2804_33_03.045348/model-00003-1.37026-0.40299-1.27944-0.50000.h5
Epoch 4/20
67/67 [==============================] - 49s 729ms/step - loss: 1.2521 - categorical_accuracy: 0.4612 - val_loss: 1.1958 - val_categorical_accuracy: 0.5400

Ep

In [25]:
  # Save model.
model_json = model.to_json()
with open("./video_classifier_conv3d_rgb_model.json", "w") as json_file:
      json_file.write(model_json)

print("json schema to disk")
# serialize weights to HDF5
model.save_weights("./video_classifier_conv3d_rgb_model.h5")
#model.save('/content/drive/My Drive/MLP/intent_classifier_mlp_model.h5')

print("Saved model to disk")

json schema to disk
Saved model to disk
